# Preprocessing

## Preparation

In [2]:
%matplotlib inline
import queue
import pyproj
import shapely
import rasterio
import threading
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
from src.utils import get_data_dir
from src.decorators import benchmark
from collections import namedtuple
from rasterio import warp, merge


DIRS = get_data_dir(str(Path('data').resolve()))
WGS84 = {'init': 'epsg:4326'}


def read_raster(item) -> rasterio.io.DatasetReader:
    """
    Helper method to return a raster file as a opened instance of
    rasterio.io.DatasetReader in read mode. Throws a exception if
    raster file is not openable with the assigned file system handle.
    
    :param item: str, pathlib.Path or rasterio.io.DatasetReader
        Should be the path to the raster file on filesystem as a string
        or pathlib.Path object. If item is a instance of DatasetReader
        the function returns immediately.
    :return: rasterio.io.DatasetReader
        Retruns an instance of rasterio.io.DatasetReader in read mode.
    """
    if isinstance(item, rasterio.io.DatasetReader):
        return item
    else:
        try:
            path = str(item)  # Cast pathlib.Path to string
            return rasterio.open(path, 'r')
        except:
            msg = 'Attr {}, Type {} is not a valid raster file'.format(item, type(item))
            raise ValueError(msg)


def fetch_metadata(features: list, from_path_or_reader: str) -> namedtuple:
    """
    This method fetches user selected metadata features from a raster file and
    returns them as a named tuple where the attribute name is the selected
    metadata feature key and the assigned value the corresponding metadata
    feature. Please refer to the documentation of rasterio for a comprehenisve 
    list of fetchable metadata features provided by a raster file.  
    
    :param features: list or tuple of str
        The requested metadata feature as a list or tuple of strings.
    :param from_path_or_reader: str, pathlib.Path or rasterio.io.DatasetReader
        Path to the raster file on drive as string or pathlib.Path object or a
        opened raster dataset.
    :return: namedtuple
        The requested metadata features as a namedtuple where the attribute 
        name is the selected metadata feature key and the assigned value the 
        corresponding metadata feature.
    Example:
        fetch_metadata(('bounds', 'crs'), path)
        (bounds=value, crs=value)
    """
    reader = read_raster(from_path_or_reader)
    
    values = []
    for f in features:
        value = reader.__getattribute__(f)
        if value is not None:
            values.append(value)
        else:
            raise ValueError('{} is not set'.format(f))
    
    # Closes the reader but if a user just want to proceed with the reader
    # provided as arg this can be a pitfal
    reader.close()
    Metadata = namedtuple('Metadata', features)
    return Metadata(*values)


# TODO refactor to def(left, right, top, y2)
def polygon_from(bounds: namedtuple) -> shapely.geometry.Polygon:
    """
    Description
    
    :param bounds: namedtuple
    :return:
    """
    x_points = ['left', 'left', 'right', 'right']
    y_points = ['top', 'bottom', 'bottom', 'top']    
    
    polygon_bounds = [
        (bounds.__getattribute__(x), bounds.__getattribute__(y))
        for x, y in zip(x_points, y_points)
    ]
    
    return shapely.geometry.Polygon(polygon_bounds)


def reproject_bounds(bounds: namedtuple, source_crs: dict, target_crs: dict) -> namedtuple:
    """
    """
    p1 = pyproj.Proj(**source_crs)
    p2 = pyproj.Proj(**target_crs)
    
    left, bottom = pyproj.transform(p1, p2, bounds.left, bounds.bottom)
    right, top = pyproj.transform(p1, p2, bounds.right, bounds.top)
    
    BoundingBox = namedtuple('BoundingBox', 'left bottom right top')
    return BoundingBox(left, bottom, right, top)


def polygoniz(paths_or_readers: list, target_crs: dict) -> gpd.GeoSeries:
    """
    """
    polygons = []
    for item in paths_or_readers:
        bounds, crs = fetch_metadata(('bounds', 'crs'), item)
        if crs != target_crs:
            bounds = reproject_bounds(bounds, crs, target_crs)
        polygon = polygon_from(bounds)
        polygons.append(polygon)
        
    geometry = gpd.GeoSeries(polygons)
    geometry.crs = target_crs
    return geometry


def tile_index(rasters: list, target_crs: dict, **kwargs) -> gpd.GeoDataFrame:
    geometry = polygoniz(rasters, target_crs)
    features = pd.DataFrame(kwargs)
    
    return gpd.GeoDataFrame(features, geometry=geometry)

## Masking

### GFC mask

In [3]:
# TODO refactor
gfc = sorted(DIRS.gfc.glob('*.tif'))

kwargs = {
    'gain': [i.name for i in gfc[:int(len(gfc)/3)]],
    'loss': [i.name for i in gfc[int(len(gfc)/3):2*int(len(gfc)/3)]],
    'cover': [i.name for i in gfc[2*int(len(gfc)/3):]],
}
gfc_mask = tile_index(gfc[:int(len(gfc)/3)], WGS84, **kwargs)
gfc_mask.to_file(str(DIRS.masks / 'gfc_mask.shp'))
gfc_mask.head()

,cover,gain,loss,geometry
0,Hansen_GFC2013_treecover2000_00N_000E.tif,Hansen_GFC2013_gain_00N_000E.tif,Hansen_GFC2013_lossyear_00N_000E.tif,POLYGON ((-0.0001388888888982365 0.00013888888...
1,Hansen_GFC2013_treecover2000_00N_010E.tif,Hansen_GFC2013_gain_00N_010E.tif,Hansen_GFC2013_lossyear_00N_010E.tif,POLYGON ((9.999861111111102 0.0001388888888840...
2,Hansen_GFC2013_treecover2000_00N_010W.tif,Hansen_GFC2013_gain_00N_010W.tif,Hansen_GFC2013_lossyear_00N_010W.tif,POLYGON ((-10.0001388888889 0.0001388888888840...
3,Hansen_GFC2013_treecover2000_00N_020E.tif,Hansen_GFC2013_gain_00N_020E.tif,Hansen_GFC2013_lossyear_00N_020E.tif,POLYGON ((19.9998611111111 0.00013888888888402...
4,Hansen_GFC2013_treecover2000_00N_020W.tif,Hansen_GFC2013_gain_00N_020W.tif,Hansen_GFC2013_lossyear_00N_020W.tif,POLYGON ((-20.0001388888889 0.0001388888888840...


### GL30 mask

- edge tiles of gl30 have coordinate system issues, x overflows boundaries of applied coor system
- result: in wgs84 polygon of a certain tile covers the entire globe

In [3]:
# TODO refactor
gl30 = sorted(DIRS.gl30.glob('*.tif'), key=lambda key: (key.name[7:11], key.name[0:6]))

gl30_00 = set(map(lambda x: x.name[0:6], gl30[:int(len(gl30)/2)]))
gl30_10 = set(map(lambda x: x.name[0:6], gl30[int(len(gl30)/2):]))
exclude = list(gl30_10 ^ gl30_00)
exclude += ['n01_00', 's01_00', 's01_10', 's01_15', 's01_20', 's60_00', 's60_05', 's60_10', 's60_15']
gl30 = [item for item in gl30 if item.name[0:6] not in exclude]

kwargs = {
    'gl30_00': [i.name for i in gl30[:int(len(gl30)/2)]],
    'gl30_10': [i.name for i in gl30[int(len(gl30)/2):]],
    'key': [i.name[0:6] for i in gl30[:int(len(gl30)/2)]]
}
gl30_mask = tile_index(gl30[int(len(gl30)/2):], WGS84, **kwargs)
gl30_mask.to_file(str(DIRS.masks / 'gl30_mask.shp'))
gl30_mask.head()

,gl30_00,gl30_10,key,geometry
0,n02_15_2000lc030.tif,n02_15_2010lc030.tif,n02_15,POLYGON ((-174.0053601744084 20.00401663536249...
1,n03_05_2000lc030.tif,n03_05_2010lc030.tif,n03_05,POLYGON ((-168.0054833302891 10.00519024901941...
2,n03_20_2000lc030.tif,n03_20_2010lc030.tif,n03_20,POLYGON ((-168.0051433812486 25.00312959291788...
3,n04_00_2000lc030.tif,n04_00_2010lc030.tif,n04_00,POLYGON ((-162.0055192236557 5.005478418984219...
4,n04_05_2000lc030.tif,n04_05_2010lc030.tif,n04_05,"POLYGON ((-162.0054833302891 10.0051902490194,..."


## Raster alignment
- store in files in a folder processed 
- reproject all files to wgs84 epsg4326 for convenience (entire gl30 dataset must be reprojected)
- intersect gl30 mask with gfc mask
- find gfc datasets covering a gl30 tile 
- merge them and crop them to the extent of gl30 tile

In [4]:
def reproject_from(in_path: str, to_crs: dict, to_out_path: str):
    # TODO accept **kwargs to alter write parameters
    with rasterio.open(in_path, 'r') as src:
        affine, width, height = rasterio.warp.calculate_default_transform(
            src_crs=src.crs,
            dst_crs=to_crs,
            width=src.width,
            height=src.height,
            **src.bounds._asdict(),
        )
        
        kwargs = src.profile.copy()
        kwargs.update(
            transform=affine,
            width=width,
            height=height,
            crs=to_crs
        )
        
        with rasterio.open(to_out_path, 'w', **kwargs) as dst:
            for idx in src.indexes:
                rasterio.warp.reproject(
                    source=rasterio.band(src, idx), 
                    destination=rasterio.band(dst, idx)
                )
        
        return to_out_path
    
    
def merge_from(paths_or_readers: list, **kwargs) -> namedtuple:
    readers = [read_raster(item) for item in paths_or_readers]

    dest, affine = rasterio.merge.merge(readers, **kwargs)
    
    [reader.close() for reader in readers]
    Merge = namedtuple('Merge', 'data affine')  
    return Merge(dest, affine)


def merge_alike(with_template: str, to_merge: list) -> namedtuple:
    bounds, res = fetch_metadata(('bounds', 'res'), with_template)
    return merge_from(to_merge, bounds=bounds, res=res)


def write(data: np.ndarray, to_path: str, **kwargs):
    if len(data.shape) == 3:
        idx, height, width = data.shape  # z, y, x
    else:
        # TODO 1D-Array need slightly different handling
        idx = 1  # z
        height, width = data.shape  # y, x
        data = np.reshape(data.copy(), (idx, height, width))
    
    dtype = data.dtype
    kwargs.update(
        count=idx,
        height=height,
        width=width,
        dtype=dtype
    )
    
    with rasterio.open(to_path, 'w', **kwargs) as dst:
        for i in range(idx):
            dst.write(data[i], i+1)  # rasterio band index start at one, thus we increment by one
    
    return to_path


def int_to_orient(x, y):
    x = round(x)
    y = round(y)
    
    lng, we = (-1 * x, 'W') if x < 0 else (x, 'E')
    lat, ns = (-1 * y, 'S') if y < 0 else (y, 'N')
    
    return '{:02d}{}_{:03d}{}'.format(lat, ns, lng, we)


def merge_worker(template, to_merge, path, **kwargs):
    data, transform = merge_alike(template, to_merge)
    
    orient = int_to_orient(transform[2], transform[5])
    to_path = path + orient + '.tif'
    
    kwargs.update(transform=transform)
    write(data, to_path, **kwargs)   


def reproject_worker(queue, **kwargs):
    path = reproject_from(kwargs['in_path'], kwargs['to_crs'], kwargs['to_out_path'])
    
    meta = fetch_metadata(features=('bounds',), from_path_or_reader=path)
    orient = int_to_orient(meta.bounds.left, meta.bounds.top)    
    name = kwargs['rename'] + orient + '.tif'
    
    src = Path(path)
    dst = Path(src.parent / name)
    src.rename(dst)
    
    queue.put(dst)

In [11]:
gfc_mask = gpd.read_file(str(DIRS.masks / 'gfc_mask.shp'))
gl30_mask = gpd.read_file(str(DIRS.masks / 'gl30_mask.shp'))

intersect = gpd.overlay(gfc_mask, gl30_mask, how='intersection')

for group in intersect.groupby(by='key', sort=False):
    print(group)

('s04_20',                                           cover  \
0     Hansen_GFC2013_treecover2000_20S_160W.tif   
1     Hansen_GFC2013_treecover2000_20S_170W.tif   
2     Hansen_GFC2013_treecover2000_20S_160W.tif   
61    Hansen_GFC2013_treecover2000_20S_160W.tif   
139   Hansen_GFC2013_treecover2000_20S_160W.tif   
140   Hansen_GFC2013_treecover2000_20S_160W.tif   
141   Hansen_GFC2013_treecover2000_20S_160W.tif   
142   Hansen_GFC2013_treecover2000_20S_160W.tif   
143   Hansen_GFC2013_treecover2000_20S_170W.tif   
159   Hansen_GFC2013_treecover2000_20S_170W.tif   
1020  Hansen_GFC2013_treecover2000_10S_160W.tif   
1022  Hansen_GFC2013_treecover2000_10S_160W.tif   
1024  Hansen_GFC2013_treecover2000_10S_170W.tif   
1026  Hansen_GFC2013_treecover2000_10S_170W.tif   
1028  Hansen_GFC2013_treecover2000_10S_170W.tif   
1030  Hansen_GFC2013_treecover2000_10S_160W.tif   
1032  Hansen_GFC2013_treecover2000_10S_160W.tif   
1202  Hansen_GFC2013_treecover2000_20S_160W.tif   
1203  Hansen_GFC2013

('s58_05',                                           cover  \
258   Hansen_GFC2013_treecover2000_00N_160E.tif   
4309  Hansen_GFC2013_treecover2000_00N_160E.tif   
4310  Hansen_GFC2013_treecover2000_00N_160E.tif   
4312  Hansen_GFC2013_treecover2000_00N_160E.tif   

                                  gain                                  loss  \
258   Hansen_GFC2013_gain_00N_160E.tif  Hansen_GFC2013_lossyear_00N_160E.tif   
4309  Hansen_GFC2013_gain_00N_160E.tif  Hansen_GFC2013_lossyear_00N_160E.tif   
4310  Hansen_GFC2013_gain_00N_160E.tif  Hansen_GFC2013_lossyear_00N_160E.tif   
4312  Hansen_GFC2013_gain_00N_160E.tif  Hansen_GFC2013_lossyear_00N_160E.tif   

                   gl30_00               gl30_10     key  \
258   s58_05_2000lc030.tif  s58_05_2010lc030.tif  s58_05   
4309  s58_05_2000lc030.tif  s58_05_2010lc030.tif  s58_05   
4310  s58_05_2000lc030.tif  s58_05_2010lc030.tif  s58_05   
4312  s58_05_2000lc030.tif  s58_05_2010lc030.tif  s58_05   

                               

735  POLYGON ((-149.9946587033328 -14.9746200864456...  )
('s07_05',                                           cover  \
419   Hansen_GFC2013_treecover2000_00N_150W.tif   
420   Hansen_GFC2013_treecover2000_00N_150W.tif   
502   Hansen_GFC2013_treecover2000_00N_150W.tif   
503   Hansen_GFC2013_treecover2000_00N_150W.tif   
504   Hansen_GFC2013_treecover2000_00N_150W.tif   
505   Hansen_GFC2013_treecover2000_00N_140W.tif   
506   Hansen_GFC2013_treecover2000_00N_140W.tif   
535   Hansen_GFC2013_treecover2000_10S_140W.tif   
536   Hansen_GFC2013_treecover2000_10S_140W.tif   
544   Hansen_GFC2013_treecover2000_10S_150W.tif   
545   Hansen_GFC2013_treecover2000_10S_150W.tif   
939   Hansen_GFC2013_treecover2000_10S_140W.tif   
940   Hansen_GFC2013_treecover2000_10S_150W.tif   
941   Hansen_GFC2013_treecover2000_10S_140W.tif   
1615  Hansen_GFC2013_treecover2000_10S_150W.tif   

                                  gain                                  loss  \
419   Hansen_GFC2013_gain_00N_150W

('s25_05',                                          cover  \
493  Hansen_GFC2013_treecover2000_00N_040W.tif   
522  Hansen_GFC2013_treecover2000_00N_030W.tif   
524  Hansen_GFC2013_treecover2000_00N_030W.tif   

                                 gain                                  loss  \
493  Hansen_GFC2013_gain_00N_040W.tif  Hansen_GFC2013_lossyear_00N_040W.tif   
522  Hansen_GFC2013_gain_00N_030W.tif  Hansen_GFC2013_lossyear_00N_030W.tif   
524  Hansen_GFC2013_gain_00N_030W.tif  Hansen_GFC2013_lossyear_00N_030W.tif   

                  gl30_00               gl30_10     key  \
493  s25_05_2000lc030.tif  s25_05_2010lc030.tif  s25_05   
522  s25_05_2000lc030.tif  s25_05_2010lc030.tif  s25_05   
524  s25_05_2000lc030.tif  s25_05_2010lc030.tif  s25_05   

                                              geometry  
493  POLYGON ((-35.99445729818344 -9.98087841506933...  
522  POLYGON ((-29.99986111111112 -9.98087841506933...  
524  POLYGON ((-29.99446575682526 -9.98087841506933...  )
('s17

('n40_20',                                           cover  \
1691  Hansen_GFC2013_treecover2000_30N_050E.tif   
1693  Hansen_GFC2013_treecover2000_30N_050E.tif   
1694  Hansen_GFC2013_treecover2000_30N_060E.tif   
1977  Hansen_GFC2013_treecover2000_30N_060E.tif   
2863  Hansen_GFC2013_treecover2000_20N_060E.tif   
2951  Hansen_GFC2013_treecover2000_20N_060E.tif   

                                  gain                                  loss  \
1691  Hansen_GFC2013_gain_30N_050E.tif  Hansen_GFC2013_lossyear_30N_050E.tif   
1693  Hansen_GFC2013_gain_30N_050E.tif  Hansen_GFC2013_lossyear_30N_050E.tif   
1694  Hansen_GFC2013_gain_30N_060E.tif  Hansen_GFC2013_lossyear_30N_060E.tif   
1977  Hansen_GFC2013_gain_30N_060E.tif  Hansen_GFC2013_lossyear_30N_060E.tif   
2863  Hansen_GFC2013_gain_20N_060E.tif  Hansen_GFC2013_lossyear_20N_060E.tif   
2951  Hansen_GFC2013_gain_20N_060E.tif  Hansen_GFC2013_lossyear_20N_060E.tif   

                   gl30_00               gl30_10     key  \
1691  n40_

('n04_15',                                           cover  \
1844  Hansen_GFC2013_treecover2000_30N_160W.tif   
1945  Hansen_GFC2013_treecover2000_30N_170W.tif   
1947  Hansen_GFC2013_treecover2000_30N_170W.tif   
1949  Hansen_GFC2013_treecover2000_30N_170W.tif   
1950  Hansen_GFC2013_treecover2000_30N_160W.tif   
1951  Hansen_GFC2013_treecover2000_30N_160W.tif   
2642  Hansen_GFC2013_treecover2000_20N_170W.tif   
2643  Hansen_GFC2013_treecover2000_20N_170W.tif   
2644  Hansen_GFC2013_treecover2000_20N_170W.tif   
2645  Hansen_GFC2013_treecover2000_20N_170W.tif   
2646  Hansen_GFC2013_treecover2000_20N_160W.tif   
2647  Hansen_GFC2013_treecover2000_20N_170W.tif   
2648  Hansen_GFC2013_treecover2000_20N_160W.tif   
2649  Hansen_GFC2013_treecover2000_20N_170W.tif   
2650  Hansen_GFC2013_treecover2000_20N_160W.tif   
2774  Hansen_GFC2013_treecover2000_20N_160W.tif   
2779  Hansen_GFC2013_treecover2000_20N_170W.tif   
2780  Hansen_GFC2013_treecover2000_20N_170W.tif   
2941  Hansen_GFC2013

('n32_10',                                           cover  \
2200  Hansen_GFC2013_treecover2000_20N_000E.tif   
2201  Hansen_GFC2013_treecover2000_20N_000E.tif   
2203  Hansen_GFC2013_treecover2000_20N_000E.tif   
2204  Hansen_GFC2013_treecover2000_20N_000E.tif   
2206  Hansen_GFC2013_treecover2000_20N_010E.tif   
2208  Hansen_GFC2013_treecover2000_20N_010E.tif   
2210  Hansen_GFC2013_treecover2000_20N_010E.tif   
2211  Hansen_GFC2013_treecover2000_20N_010E.tif   
2499  Hansen_GFC2013_treecover2000_20N_000E.tif   
2501  Hansen_GFC2013_treecover2000_20N_000E.tif   
2503  Hansen_GFC2013_treecover2000_20N_010E.tif   
2505  Hansen_GFC2013_treecover2000_20N_010E.tif   
2507  Hansen_GFC2013_treecover2000_20N_010E.tif   
2702  Hansen_GFC2013_treecover2000_20N_000E.tif   

                                  gain                                  loss  \
2200  Hansen_GFC2013_gain_20N_000E.tif  Hansen_GFC2013_lossyear_20N_000E.tif   
2201  Hansen_GFC2013_gain_20N_000E.tif  Hansen_GFC2013_lossyear

3749  POLYGON ((11.99470842842769 10.00013888888888,...  )
('n53_05',                                           cover  \
2510  Hansen_GFC2013_treecover2000_20N_130E.tif   
2511  Hansen_GFC2013_treecover2000_20N_130E.tif   
2623  Hansen_GFC2013_treecover2000_20N_130E.tif   
3267  Hansen_GFC2013_treecover2000_10N_130E.tif   
3269  Hansen_GFC2013_treecover2000_10N_130E.tif   
3451  Hansen_GFC2013_treecover2000_10N_130E.tif   
3631  Hansen_GFC2013_treecover2000_10N_130E.tif   
3758  Hansen_GFC2013_treecover2000_10N_130E.tif   
3759  Hansen_GFC2013_treecover2000_20N_130E.tif   
3760  Hansen_GFC2013_treecover2000_10N_130E.tif   
3761  Hansen_GFC2013_treecover2000_10N_130E.tif   
3762  Hansen_GFC2013_treecover2000_10N_130E.tif   
3763  Hansen_GFC2013_treecover2000_10N_130E.tif   
3764  Hansen_GFC2013_treecover2000_20N_130E.tif   
3822  Hansen_GFC2013_treecover2000_10N_130E.tif   
3828  Hansen_GFC2013_treecover2000_10N_130E.tif   

                                  gain                        

('n48_00',                                           cover  \
3212  Hansen_GFC2013_treecover2000_10N_100E.tif   
3296  Hansen_GFC2013_treecover2000_10N_100E.tif   
3336  Hansen_GFC2013_treecover2000_10N_100E.tif   
3338  Hansen_GFC2013_treecover2000_10N_100E.tif   
3339  Hansen_GFC2013_treecover2000_10N_100E.tif   
3344  Hansen_GFC2013_treecover2000_10N_100E.tif   
3345  Hansen_GFC2013_treecover2000_10N_100E.tif   
3498  Hansen_GFC2013_treecover2000_10N_100E.tif   

                                  gain                                  loss  \
3212  Hansen_GFC2013_gain_10N_100E.tif  Hansen_GFC2013_lossyear_10N_100E.tif   
3296  Hansen_GFC2013_gain_10N_100E.tif  Hansen_GFC2013_lossyear_10N_100E.tif   
3336  Hansen_GFC2013_gain_10N_100E.tif  Hansen_GFC2013_lossyear_10N_100E.tif   
3338  Hansen_GFC2013_gain_10N_100E.tif  Hansen_GFC2013_lossyear_10N_100E.tif   
3339  Hansen_GFC2013_gain_10N_100E.tif  Hansen_GFC2013_lossyear_10N_100E.tif   
3344  Hansen_GFC2013_gain_10N_100E.tif  Hansen_GF

In [5]:
# TODO refactor
gfc_mask = gpd.read_file(str(DIRS.masks / 'gfc_mask.shp'))
gl30_mask = gpd.read_file(str(DIRS.masks / 'gl30_mask.shp'))

intersect = gpd.overlay(gfc_mask, gl30_mask, how='intersection')

# iterate over gl30 tiles covered by gfc tiles
for group in intersect.groupby(by='key', sort=False).groups.values():
    tiles = intersect.iloc[group]

    # reproject gl30 tiles
    threads = []
    que = queue.Queue()
    rename = ('gl30_2000_', 'gl30_2010_')
    gl30 = list(*zip(set(tiles.gl30_00), set(tiles.gl30_10))) 
    
    for idx, item in enumerate(gl30):
        kwargs = {
            'rename': rename[idx],
            'in_path': str(DIRS.gl30 / item),
            'to_crs': WGS84,
            'to_out_path': str(DIRS.proc /item)
        }
        thread = threading.Thread(target=reproject_worker, args=(que,), kwargs=kwargs)
        thread.start()
        threads.append(thread)

    [thread.join() for thread in threads]
    template = que.get()
    
    # merge gfc tiles
    threads = []
    rename = (DIRS.proc / 'gfc_gain_', DIRS.proc / 'gfc_loss_', DIRS.proc / 'gfc_treecover_')
    gain = list(map(lambda x: DIRS.gfc / x, list(set(tiles.gain))))
    loss = list(map(lambda x: DIRS.gfc / x, list(set(tiles.loss))))
    cover = list(map(lambda x: DIRS.gfc / x, list(set(tiles.cover))))
    gfc = (gain, loss, cover)
    for idx, item in enumerate(gfc):
        kwargs = {
            'crs': WGS84,
            'driver': 'GTiff',
            'compress': 'lzw'
        }
        thread = threading.Thread(target=merge_worker, args=(template, item, str(rename[idx])), kwargs=kwargs)
        thread.start()
        threads.append(thread)
    [thread.join() for thread in threads]

## Spatial harmonization
Workflow
- consider to use additional classes from gl30 wetlands or tundra
- initial
    - select forest (class value 20) from dataset gl30 - 2000
    - recode values to binary format 20 = 1, 0 = 0
    - select forest (class value 0 - 100) from hansen tree cover 2000
    - recode values to binary format 1 - 100 = 1, 0 = 0
    - calculate Jaccard Index with chen and hansen
- looping
    - select forest (0 + 10) - 100 from hansen tree cover 2000
    - recode values to binary format (0 + 10) - 100 = 1, 0 = 0
    - calculate Jaccard Index with chen and hansen
    - do till 30 or Jaccard Index is max
Potential Images
- world agreement map with different 
    - compare chen and hansen treccover in one image
    - sum of both dataset
    - 2 = agreement, 1 = disagreement

In [14]:
def binary_jaccard(arr1, arr2, return_matrix=False):
    """
    Calculates the Jaccard Index (JI) of two equal sized binary arrays or vectors.
    If return_matrix is set to true the method provides the JI and the necessary 
    calculation matrix as a named tuple. Attention, this method does not work in-place!
    
    :param arr1, arr2: numpy.ndarray, list, tuple
        Both array alike objects sized in equal dimensions should contain exclusively 
        binary data (1,0). 
    :param return_matrix: boolean
        Optional, a boolean value determining the return of the calculation matrix. 
    :return: float OR (float, namedtuple)
        Defaultly, the method returns only the JI if, return_matrix is set to true the 
        method returns the JI and the computation matrix.
        The Matrix contains the following attributes:
        m11 = total number of attributes where arr1 == 1 and arr2 == 1
        m10 = total number of attributes where arr1 == 1 and arr2 == 0
        m01 = total number of attributes where arr1 == 0 and arr2 == 1
        m00 = not required, set to 0
    """
    A, B = np.array(arr1, dtype=np.int8), np.array(arr2, dtype=np.int8)
    
    if np.sum(np.logical_or(A<0,A>1)) != 0 or np.sum(np.logical_or(B<0,B>1)) != 0:
        raise ValueError('Attributes should contain only binary values')
  
    C = A + B
    a = (B - C) + B  # a = (A - C) + A, m10 = a == 1
    b = (A - C) + A  # b = (B - C) + B, m01 = b == 1

    # Total number of attributes where A == 1 and B == 1
    m11 = np.sum(C==2)
    # Total number of attributes where A == 1 and B == 0
    m10 = np.sum(a==-1)
    # Total number of attributes where A == 0 and B == 1
    m01 = np.sum(b==-1)
    
    jaccard = m11 / (m10 + m01 + m11)
    
    if return_matrix:
        Matrix = namedtuple('Matrix', 'm11 m10 m01 m00')
        return jaccard, Matrix(m11, m10, m01, 0)
    return jaccard


def simple_matching_coefficient(arr1, arr2, return_matrix=False):
    """
    Calculates the Simple Matching Coefficient (SMC) of two equal sized arrays or vectors.
    If return_matrix is set to true the method provides the SMC and the necessary calculation 
    matrix as a named tuple. Attention, this method does not work in-place!
    
    :param arr1, arr2: numpy.ndarray, list, tuple
        Both array alike objects sized in equal dimensions should contain exclusively 
        binary data (1,0).
    :param return_matrix: boolean
        Optional, a boolean value determining the return of the calculation matrix.
    :return: float OR (float, namedtuple)
        Defaultly, the method returns only the SMC, if return_matrix is
        set to true the method returns the SMC and the computation matrix.
        The Matrix contains the following attributes:
        m11 = total number of attributes where arr1 == 1 and arr2 == 1
        m10 = total number of attributes where arr1 == 1 and arr2 == 0
        m01 = total number of attributes where arr1 == 0 and arr2 == 1
        m00 = total number of attributes where arr1 == 0 and arr2 == 0
    """
    _, matrix = binary_jaccard(arr1, arr2, True)
    A = np.array(arr1, dtype=np.int8)
    
    # Total number of attributes where A == 0 and B == 0
    m00 = A.size - sum(matrix)
    
    smc = (matrix.m11 + m00) / A.size

    if return_matrix:
        matrix = matrix._replace(m00=m00)
        return smc, matrix
    return smc